In [2]:
import sys

import numpy as np
from process_bigraph import pp


sys.path.insert(0, '..')


from biosimulator_processes.services.rest_service import BiosimulationsRestService

/Users/alexanderpatrie/Library/Caches/pypoetry/virtualenvs/biosimulator-processes-JaN5cQMh-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CobraProcess registered successfully.
CopasiProcess registered successfully.
_CopasiProcess registered successfully.
SmoldynProcess registered successfully.
TelluriumProcess registered successfully.
AmiciProcess registered successfully.
ODEComparatorProcess not available. Error: cannot import name 'fetch_sbml_file' from 'biosimulator_processes.io' (/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/demos/../biosimulator_processes/io.py)
GetSbmlStep not available. Error: cannot import name 'fetch_sbml_file' from 'biosimulator_processes.io' (/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/demos/../biosimulator_processes/io.py)
CompositionPlotter registered successfully.
Plotter2d registered successfully.
ODEComparatorStep registered successfully.


In [3]:
repressilator_run_id = "61fea4a08c1e3dc95a79802e"

query = "tumor"

result = await BiosimulationsRestService().extract_data(query)

Failed to fetch simulation run files:
Expecting value: line 1 column 1 (char 0)


AttributeError: 'NoneType' object has no attribute 'run_id'

In [4]:
import requests
get_files_url = f'https://api.biosimulations.dev/results/{repressilator_run_id}/download'
headers = {'accept': 'application/json'}
files_resp = requests.get(get_files_url, headers=headers)

In [8]:
files_resp.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [9]:
import h5py

def explore_hdf5(file_path):
    with h5py.File(file_path, 'r') as f:
        def print_attrs(name, obj):
            print(f"Name: {name}")
            print("Attributes:")
            for key, val in obj.attrs.items():
                print(f"    {key}: {val}")
            if isinstance(obj, h5py.Dataset):
                print(f"Dataset shape: {obj.shape}")
                print(f"Dataset dtype: {obj.dtype}")
            elif isinstance(obj, h5py.Group):
                print("Group")
            print()
        
        f.visititems(print_attrs)

# Usage example
file_path = '/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/test_suite/examples/sbml-core/Elowitz-Nature-2000-Repressilator/reports.h5'
explore_hdf5(file_path)

Name: simulation.sedml
Attributes:
    combineArchiveLocation: simulation.sedml
    uri: simulation.sedml
Group

Name: simulation.sedml/Figure_1c
Attributes:
    _type: SedPlot2D
    sedmlDataSetDataTypes: ['float64' 'float64' 'float64' 'float64']
    sedmlDataSetIds: ['data_gen_px' 'data_gen_py' 'data_gen_pz' 'data_gen_time']
    sedmlDataSetLabels: ['data_gen_px' 'data_gen_py' 'data_gen_pz' 'data_gen_time']
    sedmlDataSetNames: ['LacI protein' 'TetR protein' 'cI protein' 'time']
    sedmlDataSetShapes: ['601' '601' '601' '601']
    sedmlId: Figure_1c
    sedmlName: Figure 1c
    uri: simulation.sedml/Figure_1c
Dataset shape: (4, 601)
Dataset dtype: float64

Name: simulation.sedml/report
Attributes:
    _type: SedReport
    sedmlDataSetDataTypes: ['float64' 'float64' 'float64' 'float64' 'float64' 'float64' 'float64']
    sedmlDataSetIds: ['data_set_time' 'data_set_laci_protein' 'data_set_tetr_protein'
 'data_set_ci_protein' 'data_set_laci_mrna' 'data_set_tetr_mrna'
 'data_set_ci_mrn

In [12]:
def read_dataset(file_path, dataset_id):
    with h5py.File(file_path, 'r') as f:
        if dataset_id in f:
            data = f[dataset_id][:]
            print(f"Data from dataset '{dataset_id}':\n{data}")
        else:
            print(f"Dataset '{dataset_id}' not found in the file.")
            
            
read_dataset(file_path, 'data_set_time')

Dataset 'data_set_time' not found in the file.


In [30]:
import h5py
from dataclasses import dataclass



    

def read_report_outputs(report_file_path, dataset_label, group_path="simulation.sedml/report") -> BiosimulationsRunOutput:
    """Read the outputs from all species in the given report file from biosimulations output.
        
        Args:
            report_file_path (str): The path to the simulation.sedml/report.h5 HDF5 file.
            dataset_label (str): The `<dataSet label>` value for the species from the SEDML file.
            group_path (str): The path to the simulation.sedml/report.h5 HDF5 file. Defaults to `simulation.sedml/report`
    
    """
    outputs = []
    with h5py.File(report_file_path, 'r') as f:
        if group_path in f:
            group = f[group_path]
            dataset_labels = group.attrs['sedmlDataSetLabels']
            for label in dataset_labels:
                dataset_index = list(dataset_labels).index(label)
                data = group[()]
                
                specific_data = data[dataset_index]
                output = BiosimulationsSpeciesOutput(dataset_label=label, data=specific_data)
                outputs.append(output)
            return BiosimulationsRunOutput(report_path=report_file_path, data=outputs)
        else:
            print(f"Group '{group_path}' not found in the file.")

In [31]:
group_path = 'simulation.sedml/report'
dataset_label = 'cI protein'
read_specific_species_dataset(file_path, dataset_label)

BiosimulationsRunOutput(report_path='/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/test_suite/examples/sbml-core/Elowitz-Nature-2000-Repressilator/reports.h5', data=[BiosimulationsSpeciesOutput(dataset_label='Time', data=array([ 400.,  401.,  402.,  403.,  404.,  405.,  406.,  407.,  408.,
        409.,  410.,  411.,  412.,  413.,  414.,  415.,  416.,  417.,
        418.,  419.,  420.,  421.,  422.,  423.,  424.,  425.,  426.,
        427.,  428.,  429.,  430.,  431.,  432.,  433.,  434.,  435.,
        436.,  437.,  438.,  439.,  440.,  441.,  442.,  443.,  444.,
        445.,  446.,  447.,  448.,  449.,  450.,  451.,  452.,  453.,
        454.,  455.,  456.,  457.,  458.,  459.,  460.,  461.,  462.,
        463.,  464.,  465.,  466.,  467.,  468.,  469.,  470.,  471.,
        472.,  473.,  474.,  475.,  476.,  477.,  478.,  479.,  480.,
        481.,  482.,  483.,  484.,  485.,  486.,  487.,  488.,  489.,
        490.,  491.,  492.,  493.,  494.,  495.,  496.,  497.,  4